In [1]:
import logging
logging.basicConfig(level=logging.INFO)

log = logging.getLogger('')
rlylog = log.getChild('relay')
mqlog = log.getChild('mqtt')

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [ ]:
from onvif import ONVIFCamera
from onvif.exceptions import ONVIFError
import datetime

In [ ]:
camparam = {
    'onvif': {
        'host': '10.0.2.5',
        'port': 80,
        'user': '',
        'passwd': '',
    },
    'limits': {
        'az_min': -180,
        'az_max': 180,
        'az_off': 180,
        'el_min': 90,
        'el_max': 0,
        'el_off': 15,
        'z_min': 0,
        'z_max': 1,
        'z_off': 0,
    },
}

In [ ]:
camera = ONVIFCamera(**camparam['onvif'])

In [ ]:
limits = dotdict(camparam['limits'])

In [ ]:
camera.devicemgmt.GetCapabilities()

In [ ]:
camera.create_ptz_service()

In [ ]:
camera.ptz.GetServiceCapabilities()

In [3]:
from osrd_packet import *

In [4]:
import serial

In [5]:
ser = serial.Serial('COM23', 9600)

In [6]:
ser

Serial<id=0x2a3877f7080, open=True>(port='COM23', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)

In [9]:
len_cs(int_to_7bit(0, 2) + b'\x0A')

b'\x84\x00\x00\n\x0e'

In [40]:
ser.write(len_cs(int_to_7bit(0, 2) + b'\x0A'))

5

In [41]:
ser.read_all()

b'\x8a\x00\x00\n\r6\x00\x07Y\x007'

In [46]:
ser.read_all()

b''

In [50]:
create_osrd_pt(0, 0, 0).hex().upper()

'8A00000A00000000000014'

In [47]:
ser.close()

In [49]:
len_cs(int_to_7bit(0, 2) + b'\x0A').hex()

'8400000a0e'

In [51]:
import requests

In [62]:
az = 0
el = 0

In [64]:
requests.get(
    'http://10.0.2.5/rcp.xml?command=0x09A5&type=P_OCTET&direction=WRITE&num=1&payload=0x80000601d002' 
            + az.to_bytes(4,'big').hex().upper()
            + el.to_bytes(4,'big').hex().upper())

<Response [200]>

In [55]:
?az.to_bytes

In [ ]:
"""
payload: 
80 (flags)
0006 (bicom ptz server)
one of:
    01d0 (PTZPosMoveStatus)
    0112 (Position.Pan)
    0113 (Position.Tilt)
    0114 (Position.ZoomTicks)
02 (operation = set)

for Position.pan, tilt, or zoom
    4-bytes pan in rads * 10000
    4-bytes tilt in rads * 10000
    4-bytes zoom in ticks from 0 to 255

for PTZPosMoveStatus or AutoTracker.AT
    2-bytes pan in degree * 100
    2-bytes tilt in degree * 100
    2-bytes zoom in focal length * 100
    1-byte ignore mask?
    1-byte don't care
    1-byte CRC
"""